In [1]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
from src.data_loader import FairFaceData, CelebData, EmbeddingData
from src.models import transformation, get_pretrained_model

import torch.utils.data as torchdata
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path


c:\Users\johnd\anaconda3\envs\computer_vision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup Data Storage Paths

In [2]:
# Define Paths
root = Path('..')
preproc_root = root / 'data_processed'
preproc_root.mkdir(exist_ok=True)

# Setup Models

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


# Preprocess Embeddings for Celeb Classification 

We use these embeddings for determining an appropriate classification model.

In [4]:
# we only want individuals without eyglasses
celeb_data = CelebData(root = str(root), sample = False)

In [5]:
celeb_data.get_all_filter()

Index(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'],
      dtype='object')

In [6]:
celeb_data_ng = celeb_data.filter_dataset({'Eyeglasses': -1})

In [7]:
len(celeb_data_ng)

189406

In [8]:
celeb_dataloader = torchdata.DataLoader(celeb_data_ng, batch_size=128, shuffle=False)

In [9]:
def compute_embeddings(model, dataloader, save: Path = None):
    ''' computes embeddings & results as 0-1 instead of -1 to 1'''
    model.to(device)
    model.eval()

    with torch.no_grad():
        d = enumerate(dataloader)
        idx, (img, gender)  = next(d)
        img = img.float().to(device)
        # outputs logit tensors.
        total = model(img)
        rez = ((gender + 1)/2).int()
        # iterate through all images and compute a euclidian
        for idx, (img, gender) in tqdm(d, total = len(dataloader) -1):
            # move img to device
            img = img.to(device)
            
            # outputs logit tensors.
            t = model(img)
            r = ((gender + 1)/2).int()
            
            # combine with current total
            total = torch.vstack([total, t])
            rez = torch.vstack([rez, r])
    
    if save:
        torch.save(total, save / 'embeddings.pt')
        torch.save(rez, save / 'gender.pt')
    del total, rez
        
    return

Model 1 - Facial Recognition

In [54]:
model1 = get_pretrained_model('vggface2', device)

In [56]:
celeb_embeddings = preproc_root / 'celeb_embeddings_vggface2'
celeb_embeddings.mkdir(exist_ok=True, parents=True)

In [57]:
# modify dataset transforms
celeb_data.set_transformation(transformation('vggface2'))

In [58]:
# save full
compute_embeddings(model1, celeb_dataloader, save = celeb_embeddings)

100%|██████████| 1479/1479 [08:13<00:00,  3.00it/s]


In [59]:
# check it worked
test = EmbeddingData(data_dir_name = 'celeb_embeddings_vggface2', root = str(root), device = device, sample = False)
len(test[2][0]), len(test[2][1])

(512, 1)

In [69]:
del model1

Model 2 - Facial Recognition

In [60]:
model2 = get_pretrained_model('casia', device)

In [61]:
celeb_embeddings = preproc_root / 'celeb_embeddings_casia'
celeb_embeddings.mkdir(exist_ok=True, parents=True)

In [63]:
# modify transforms
celeb_data.set_transformation(transformation('casia'))

In [64]:
# save full
compute_embeddings(model2, celeb_dataloader, save = celeb_embeddings)

100%|██████████| 1479/1479 [04:58<00:00,  4.96it/s]


In [70]:
del model2

Model 3 - VGG16, Imagenet

In [10]:
model3 = get_pretrained_model('vgg16', device)

In [11]:
celeb_embeddings = preproc_root / 'celeb_embeddings_vgg16'
celeb_embeddings.mkdir(exist_ok=True, parents=True)

In [12]:
celeb_data.set_transformation(transformation('vgg16'))

In [13]:
compute_embeddings(model3, celeb_dataloader, save = celeb_embeddings)

100%|██████████| 1479/1479 [07:17<00:00,  3.38it/s]


# Preprocess Embeddings for different Ethnicities. 

In [24]:
def compute_embeddings_fface(model, dataloader, type ,save: Path = None):
    ''' computes embeddings & results as 0-1'''
    model.to(device)
    model.eval()

    with torch.no_grad():
        d = enumerate(dataloader)
        idx, (img, gender)  = next(d)
        img = img.to(device)
        # outputs logit tensors.
        total = model(img)
        rez = gender
        # iterate through all images and compute a euclidian
        for idx, (img, gender) in tqdm(d, total = len(dataloader) -1):
            # move img to device
            img = img.to(device)
            
            # outputs logit tensors.
            t = model(img)
            r = gender
            
            # combine with current total
            total = torch.vstack([total, t])
            rez = torch.vstack([rez, r])
    
    if save:
        torch.save(total, save / f'{type}_embeddings.pt')
        torch.save(rez, save / f'{type}_gender.pt')
    del total, rez
        
    return

In [25]:
fface = FairFaceData(root = str(root), sample = False)

In [26]:
fface.set_filter(['gender'])

In [27]:
fface.get_all_filter()

Index(['age', 'gender', 'race', 'service_test'], dtype='object')

In [28]:
fface.get_attr_map()

{'age': {'50-59': 0,
  '30-39': 1,
  '3-9': 2,
  '20-29': 3,
  '40-49': 4,
  '10-19': 5,
  '60-69': 6,
  '0-2': 7,
  'more than 70': 8},
 'gender': {'Male': 1, 'Female': 0},
 'race': {'East Asian': 0,
  'Indian': 1,
  'Black': 2,
  'White': 3,
  'Middle Eastern': 4,
  'Latino_Hispanic': 5,
  'Southeast Asian': 6},
 'service_test': {True: 0, False: 1}}

In [18]:
# process by ethnicity

inddata = fface.filter_dataset(filter_={'race': 'Indian'})

blackdata = fface.filter_dataset(filter_={'race': 'Black'})
medata = fface.filter_dataset(filter_={'race': 'Middle Eastern'})
lhdata = fface.filter_dataset(filter_={'race': 'Latino_Hispanic'})
sedata = fface.filter_dataset(filter_={'race': 'Southeast Asian'})
datasets = [eadata, inddata, whitedata, blackdata, medata, lhdata, sedata]

In [29]:
# create dir
fface_embeddings = preproc_root / 'fface_embeddings_vggface2'
fface_embeddings.mkdir(exist_ok=True, parents=True)

In [30]:
# load vggface2 version
fface.set_transformation(transformation('vggface2'))

In [31]:
model_fface = get_pretrained_model('vggface2', device)

East Asian

In [32]:
eadata = fface.filter_dataset(filter_={'race': 'East Asian'})
ealoader = torchdata.DataLoader(eadata, batch_size=128, shuffle=False)

In [33]:
compute_embeddings_fface(model_fface, ealoader, type = 'ea', save = fface_embeddings)

100%|██████████| 108/108 [01:16<00:00,  1.42it/s]


Indian

In [34]:
inddata = fface.filter_dataset(filter_={'race': 'Indian'})
indloader = torchdata.DataLoader(inddata, batch_size=128, shuffle=False)

In [35]:
compute_embeddings_fface(model_fface, indloader, type = 'ind', save = fface_embeddings)

100%|██████████| 108/108 [01:15<00:00,  1.43it/s]


White

In [38]:
whitedata = fface.filter_dataset(filter_={'race': 'White'})
whiteloader = torchdata.DataLoader(whitedata, batch_size=128, shuffle=False)

In [40]:
compute_embeddings_fface(model_fface, whiteloader, type = 'white', save = fface_embeddings)

100%|██████████| 145/145 [01:13<00:00,  1.96it/s]


Black

In [41]:
blackdata = fface.filter_dataset(filter_={'race': 'Black'})
blackloader = torchdata.DataLoader(blackdata, batch_size=128, shuffle=False)

In [42]:
compute_embeddings_fface(model_fface, blackloader, type = 'black', save = fface_embeddings)

100%|██████████| 107/107 [01:12<00:00,  1.47it/s]


Middle Eastern

In [43]:
medata = fface.filter_dataset(filter_={'race': 'Middle Eastern'})
meloader = torchdata.DataLoader(medata, batch_size=128, shuffle=False)

In [44]:
compute_embeddings_fface(model_fface, meloader, type = 'me', save = fface_embeddings)

100%|██████████| 81/81 [00:55<00:00,  1.45it/s]


Latino/Hispanic

In [45]:
lhdata = fface.filter_dataset(filter_={'race': 'Latino_Hispanic'})
lhloader = torchdata.DataLoader(lhdata, batch_size=128, shuffle=False)

In [46]:
compute_embeddings_fface(model_fface, lhloader, type = 'lh', save = fface_embeddings)

100%|██████████| 117/117 [01:16<00:00,  1.53it/s]


Southeast Asian

In [47]:
sedata = fface.filter_dataset(filter_={'race': 'Southeast Asian'})
seloader = torchdata.DataLoader(sedata, batch_size=128, shuffle=False)

In [48]:
compute_embeddings_fface(model_fface, seloader, type = 'se', save = fface_embeddings)

100%|██████████| 95/95 [01:03<00:00,  1.50it/s]
